<a href="https://colab.research.google.com/github/GreatLearningAIML1/test-repository2-Hamshavarthini/blob/master/Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt 
%matplotlib inline
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')



import re
import string
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import operator
from gensim.models import KeyedVectors
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
from tqdm import tqdm
tqdm.pandas()
import pickle
import gc

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
Job_Description= pd.read_csv('/content/gdrive/My Drive/Capstone/indeed_job_dataset_1.csv')

In [0]:
Job_Description.head(2)

,Job_Title,Job_Type,Skill,No_of_Skills,Company,No_of_Reviews,No_of_Stars,Date_Since_Posted,Description,Location,Company_Industry
0,Data Scientist,data_scientist,"SAP, SQL",2,Express Scripts,3301.0,3.3,1.0,"POSITION SUMMARY, \r\r\nThe Business Analyst r...",MO,Health Care
1,Data Scientist,data_scientist,"Machine Learning, R, SAS, SQL, Python",5,Money Mart Financial Services,NaN,NaN,15.0,"What do we need?, \r\r\nYou to have an amazing...",TX,NaN


#TEXT CLEANING

In [0]:
# remove space
spaces = ['\u200b', '\u200e', '\u202a', '\u202c', '\ufeff', '\uf0d8', '\u2061', '\x10', '\x7f', '\x9d', '\xad', '\xa0']
def remove_space(text):
    """
    remove extra spaces and ending space if any
    """
    for space in spaces:
        text = text.replace(space, ' ')
    text = text.strip()
    text = re.sub('\s+', ' ', text)
    return text

In [0]:
Job_Description['clean_text'] = Job_Description['Description'].apply(lambda x:remove_space(str(x)))

In [0]:
# replace strange punctuations and raplace diacritics
from unicodedata import category, name, normalize

def remove_diacritics(s):
    return ''.join(c for c in normalize('NFKD', s.replace('ø', 'o').replace('Ø', 'O').replace('⁻', '-').replace('₋', '-'))
                  if category(c) != 'Mn')

special_punc_mappings = {"—": "-", "–": "-", "_": "-", '”': '"', "″": '"', '“': '"', '•': '.', '−': '-',
                         "’": "'", "‘": "'", "´": "'", "`": "'", '\u200b': ' ', '\xa0': ' ','،':'','„':'',
                         '…': ' ... ', '\ufeff': ''}
def clean_special_punctuations(text):
    for punc in special_punc_mappings:
        if punc in text:
            text = text.replace(punc, special_punc_mappings[punc])
    # 注意顺序，remove_diacritics放前面会导致 'don´t' 被处理为 'don t'
    text = remove_diacritics(text)
    return text

In [0]:
Job_Description['clean_text'] = Job_Description['clean_text'].apply(lambda x:remove_diacritics(str(x)))

In [0]:
# de-contract the contraction
def decontracted(text):
    # specific
    text = re.sub(r"(W|w)on(\'|\’)t ", "will not ", text)
    text = re.sub(r"(C|c)an(\'|\’)t ", "can not ", text)
    text = re.sub(r"(Y|y)(\'|\’)all ", "you all ", text)
    text = re.sub(r"(Y|y)a(\'|\’)ll ", "you all ", text)

    # general
    text = re.sub(r"(I|i)(\'|\’)m ", "i am ", text)
    text = re.sub(r"(A|a)in(\'|\’)t ", "is not ", text)
    text = re.sub(r"n(\'|\’)t ", " not ", text)
    text = re.sub(r"(\'|\’)re ", " are ", text)
    text = re.sub(r"(\'|\’)s ", " is ", text)
    text = re.sub(r"(\'|\’)d ", " would ", text)
    text = re.sub(r"(\'|\’)ll ", " will ", text)
    text = re.sub(r"(\'|\’)t ", " not ", text)
    text = re.sub(r"(\'|\’)ve ", " have ", text)
    return text

In [0]:
Job_Description['clean_text'] = Job_Description['clean_text'].apply(lambda x:decontracted(str(x)))

In [0]:
regular_punct = list(string.punctuation)
extra_punct = [
    ',', '.', '"', ':', ')', '(', '!', '?',' ?', ' !' ,'|', ';', "'", '$', '&',
    '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
    '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',
    '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '“', '★', '”',
    '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾',
    '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼', '⊕', '▼',
    '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲',
    'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»',
    '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø',
    '¹', '≤', '‡', '√', '«', '»', '´', 'º', '¾', '¡', '§', '£', '₤']
all_punct = list(set(regular_punct + extra_punct))
# do not spacing - and .
all_punct.remove('-')
all_punct.remove('.')
all_punct.remove('!')
all_punct.remove('?')

def spacing_punctuation(text):
    """
    add space before and after punctuation and symbols
    """
    for punc in all_punct:
        if punc in text:
            text = text.replace(punc, f' {punc} ')
    return text

In [0]:
Job_Description['clean_text'] = Job_Description['clean_text'].apply(lambda x:spacing_punctuation(str(x)))

In [0]:
Job_Description['clean_text'][1]

'What do we need? ,  You to have an amazing personality and communication style. That you are super-organized and are a problem solver. That you take pride in everything that you do ,  and it shows. And most importantly that you have unquestionable integrity. ,  Why work for us? ,  We invest in our employees ,  and offer extensive training ,  and development programs to set you up for future success. ,  If we sound like a fit ,  and you are ready to start an exciting career with an organization that fosters employee growth ,  apply today! Job Description This position will be responsible for :  ,  Develop marketing ,  credit risk models ,  segmentation ,  and risk analytics Perform all required tests and measures of developed models Deliver comprehensive model documentation  ( e.g. ,  Model Development Document ,  Technical Specification Document )  of modeling projects Understand and follow modeling procedures ,  credit policies ,  and deliver technical / regulatory documentation for 

In [0]:
# job_df= pd.DataFrame(Job_Description['clean_text'])

In [0]:
# filename = 'job_Desciption_semi_clean_1.csv'

# job_df.to_csv(filename,index=False)

# print('Saved file: ' + filename)